# Supervised Fine-Tuning (SFT) at scale with DeepSpeed

This guide provides a step-by-step workflow for supervised fine-tuning the [`Qwen/Qwen2.5-32B-Instruct`](https://huggingface.co/Qwen/Qwen2.5-32B-Instruct) model on a multi-GPU Anyscale cluster. You use LLaMA-Factory for the training framework and `DeepSpeed` to efficiently manage memory and scale the training process.

SFT is a technique to adapt a pre-trained model to specific tasks. By showing the model high-quality examples of instructions and their desired outputs, you teach it to follow new instructions more accurately.

## Step 1: Set up your environment

### Dependencies
First, ensure your environment has the correct libraries. Start with a pre-built container image and install LLaMA-Factory and DeepSpeed on top of it.

Recommended container image:
```bash
anyscale/ray-llm:2.48.0-py311-cu128
```

Execute the following commands to install the required packages and optional tools for experiment tracking and faster model downloads:

In [1]:
%%bash
# Install the specific version of LLaMA-Factory
pip install -q llamafactory==0.9.3

# Install DeepSpeed for large-scale training
pip install -q deepspeed==0.16.9

# (Optional) For experiment tracking with Weights & Biases
pip install -q wandb==0.21.3

# (Optional) For accelerated model downloads from Hugging Face
pip install -q hf_transfer==0.1.9

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.12.1 which is incompatible.


Successfully registered `llamafactory` package to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_a6j8iubw9rqbyigfwk9fut4amk/prj_a8aurpnjjkhushuarbyy4kwkre/workspaces/expwrk_kpm6l9gjz6gdcskt2zb8i3fie6?workspace-tab=dependencies
Successfully registered `deepspeed` package to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_a6j8iubw9rqbyigfwk9fut4amk/prj_a8aurpnjjkhushuarbyy4kwkre/workspaces/expwrk_kpm6l9gjz6gdcskt2zb8i3fie6?workspace-tab=dependencies
Successfully registered `wandb` package to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_a6j8iubw9rqbyigfwk9fut4amk/prj_a8aurpnjjkhushuarbyy4kwkre/workspaces/expwrk_kpm6l9gjz6gdcskt2zb8i3fie6?workspace-tab=dependencies
Successfully registered `hf_transfer` package to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_a6j8iubw9

### Model and compute resources

DeepSpeed ZeRO-3 partitions parameters, gradients, and optimizer states across multiple GPUs, enabling supervised fine-tuning (SFT) of 30B+ LLMs on just 4 GPUs.

| Item | Value |
|------|-------|
| **Base model** | [`Qwen/Qwen2.5-32B-Instruct`](https://huggingface.co/Qwen/Qwen2.5-32B-Instruct) |
| **Worker Nodes** | 4 × L40S / 4 x A100-40G |

## Step 2: Prepare the dataset

### Understand the dataset
This tutorial uses [`glaive_toolcall_en_demo`](https://huggingface.co/datasets/zuol/glaive_toolcall_en_demo/tree/main), a dataset designed to teach models how to use tools (also known as function calling).

This dataset contains conversational examples where the model needs to interact with external tools. Each entry includes:
* `conversations`: A turn-by-turn log between a human and the gpt assistant.
* `tools`: A JSON schema describing the functions the model can call.

**Note**: The `conversations` may include special turns like function_call (the model deciding to call a tool) and observation (the result returned from the tool). This structure is ideal for teaching the model sophisticated tool-use behavior. To maintain role alignment in ShareGPT format, you must follow a strict turn order: `human` and `observation` (tool output) must appear in odd-numbered positions, while `gpt` and `function_call` must appear in even-numbered positions. The model learns to generate the content in the `gpt` and `function_call` turns.

**Dataset example**

```json
{
"conversations": [
    {
    "from": "human",
    "value": "Hi, I have some ingredients and I want to cook something. Can you help me find a recipe?"
    },
    {
    "from": "gpt",
    "value": "Of course! I can help you with that. Please tell me what ingredients you have."
    },
    {
    "from": "human",
    "value": "I have chicken, bell peppers, and rice."
    },
    {
    "from": "function_call",
    "value": "{\"name\": \"search_recipes\", \"arguments\": {\"ingredients\": [\"chicken\", \"bell peppers\", \"rice\"]}}"
    },
    {
    "from": "observation",
    "value": "{\"recipes\": [{\"name\": \"Chicken and Bell Pepper Stir Fry\", \"ingredients\": [\"chicken\", \"bell peppers\", \"rice\"], \"instructions\": \"Cut the chicken into small pieces. Slice the bell peppers. Cook the rice. Stir fry the chicken and bell peppers. Serve over rice.\"}, {\"name\": \"Chicken and Rice Casserole\", \"ingredients\": [\"chicken\", \"bell peppers\", \"rice\"], \"instructions\": \"Cook the chicken and rice separately. Mix them together with the bell peppers in a casserole dish. Bake until golden brown.\"}]}"
    },
    {
    "from": "gpt",
    "value": "I found two recipes for you. The first one is \"Chicken and Bell Pepper Stir Fry\". The instructions are: Cut the chicken into small pieces. Slice the bell peppers. Cook the rice. Stir fry the chicken and bell peppers. Serve over rice. The second one is \"Chicken and Rice Casserole\". The instructions are: Cook the chicken and rice separately. Mix them together with the bell peppers in a casserole dish. Bake until golden brown. Which one would you like to try?"
    },
    {
    "from": "human",
    "value": "They both sound delicious, but I think I'll try the stir fry. Can you order the ingredients for me?"
    },
    {
    "from": "gpt",
    "value": "I'm sorry, but as an AI, I don't have the capability to perform external tasks such as ordering ingredients. However, I can help you find more recipes or provide cooking tips if you need."
    }
],
"tools": "[{\"name\": \"search_recipes\", \"description\": \"Search for recipes based on ingredients\", \"parameters\": {\"type\": \"object\", \"properties\": {\"ingredients\": {\"type\": \"array\", \"items\": {\"type\": \"string\"}, \"description\": \"The ingredients to search for\"}}, \"required\": [\"ingredients\"]}}]"
}
```

### Register the dataset

To specify new datasets that are accessible across Ray worker nodes, you must first add a **`dataset_info.json`** to **[storage shared across nodes](https://docs.anyscale.com/configuration/storage#shared)** such as `/mnt/cluster_storage`. This configuration file acts as a central registry for all your datasets. It maps a custom name to your dataset file location, format, and column structure. 

If you plan to run SFT fine-tuning on the `glaive_toolcall_en_demo` dataset, first complete the setup steps below. Ensure that you place the dataset files in a storage location that all workers can access (for example, a shared mount or object storage). Avoid storing large files on the head node.

`dataset_info.json`
```json
{
  "my_glaive_toolcall_en_demo": {
      "file_name": "/mnt/cluster_storage/glaive_toolcall_en_demo.json",
      "formatting": "sharegpt",
      "columns": {
          "messages": "conversations",
          "tools": "tools"
      }
  }
}
```

For a more detailed dataset preparation and formatting guide, see [Choose your data format](https://docs.anyscale.com/llm/fine-tuning/data-preparation#sft).

In [2]:
%%bash
# Make sure all files are accessible to worker nodes
# Create a copy of the data in /mnt/cluster_storage
wget https://anyscale-public-materials.s3.us-west-2.amazonaws.com/llm-finetuning/llama-factory/datasets/sharegpt/glaive_toolcall_en_demo.json -O /mnt/cluster_storage/glaive_toolcall_en_demo.json
# Create a copy of the dataset registry in /mnt/cluster_storage
cp ../dataset-configs/dataset_info.json /mnt/cluster_storage/

--2026-02-08 08:44:10--  https://anyscale-public-materials.s3.us-west-2.amazonaws.com/llm-finetuning/llama-factory/datasets/sharegpt/glaive_toolcall_en_demo.json
Resolving anyscale-public-materials.s3.us-west-2.amazonaws.com (anyscale-public-materials.s3.us-west-2.amazonaws.com)... 3.5.85.37, 52.92.130.162, 52.92.177.58, ...
Connecting to anyscale-public-materials.s3.us-west-2.amazonaws.com (anyscale-public-materials.s3.us-west-2.amazonaws.com)|3.5.85.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 738925 (722K) [application/json]
Saving to: ‘/mnt/cluster_storage/glaive_toolcall_en_demo.json’

     0K .......... .......... .......... .......... ..........  6%  325K 2s
    50K .......... .......... .......... .......... .......... 13%  325K 2s
   100K .......... .......... .......... .......... .......... 20%  380K 2s
   150K .......... .......... .......... .......... .......... 27% 2.19M 1s
   200K .......... .......... .......... .......... .......... 34%

## Step 3: Create the fine-tuning config (SFT with DeepSpeed)

Next, create the main YAML configuration file—the master recipe for your fine-tuning job. It specifies the base model, the fine-tuning method (LoRA), the dataset, training hyperparameters, cluster resources, and more.

**Important notes:**
- **W&B tracking:** To track experiments with Weights & Biases (W&B), set `report_to: wandb` in the config and provide `WANDB_API_KEY` in the runtime environment. If you don't want to use W&B, set `report_to: none` to avoid errors.
- **Access and paths:** The YAML only needs to be on the **head node**, but any referenced paths (`dataset_dir`, `output_dir`) must reside on storage **reachable by all workers** (for example, `/mnt/cluster_storage/`).
- **Gated models:** If your base model has gated access (for example, Llama) on Hugging Face, set `HF_TOKEN` in the runtime environment.
- **GPU selection and placement:** The config uses a 4xL40S node (`ananyscale/accelerator_shape:4xL40S`) so that all 4 GPUs are on the same machine, which is important for efficient DeepSpeed ZeRO-3 communication. You can switch to other multi-GPU nodes such as `4xA100-40GB` or any other node type with comparable or more VRAM, depending on your cloud availability.

### Configure LLaMA-Factory with Ray

**Note**: To customize the training configuration, edit `train-configs/sft_lora_deepspeed.yaml`. 

```yaml
# sft_lora_deepspeed.yaml

### model
model_name_or_path: Qwen/Qwen2.5-32B-Instruct
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 8
lora_target: all

### deepspeed
deepspeed: /mnt/cluster_storage/ds_z3_config.json # path to the DeepSpeed config

### dataset
dataset: my_glaive_toolcall_en_demo
dataset_dir: /mnt/cluster_storage

template: qwen
cutoff_len: 1024
max_samples: 1000
overwrite_cache: true
preprocessing_num_workers: 16

### output
output_dir: qwen2.5_32b_lora_sft
logging_steps: 5
save_steps: 50
plot_loss: true
report_to: wandb # or none

### train
per_device_train_batch_size: 1 # Adjust this depending on your GPU memory and sequence length
gradient_accumulation_steps: 4
num_train_epochs: 3.0
learning_rate: 1.0e-4
bf16: true
lr_scheduler_type: cosine
warmup_ratio: 0.1
ddp_timeout: 180000000

### ray
ray_run_name: qwen2.5_32b_lora_sft
ray_storage_path: /mnt/cluster_storage/
ray_num_workers: 4  # Number of GPUs to use
resources_per_worker:
  GPU: 1
  # accelerator_type:L40S: 0.001            # Use this to simply specify a GPU type (may place GPUs on separate nodes).
  anyscale/accelerator_shape:4xL40S: 0.001  # Prefer this for DeepSpeed so all 4 GPUs are on the same node.
  # See https://docs.ray.io/en/master/ray-core/accelerator-types.html#accelerator-types for a full list of accelerator types.
ray_init_kwargs:
  runtime_env:
    env_vars:
      # If using wandb for experiments tracking
      WANDB_API_KEY: <your_wandb_token>
      # If using gated models like meta-llama/Llama-3.1-8B-Instruct
      # HF_TOKEN: <your_huggingface_token>
      # If hf_transfer is installed
      HF_HUB_ENABLE_HF_TRANSFER: '1'
```

**Note:**
This configuration assumes `4xL40S` GPUs are available in your cloud environment. If not, you can substitute with `4xA100-40G` (or another supported accelerator with similar VRAM).

### DeepSpeed configuration
DeepSpeed is an open-source deep-learning optimization library developed by Microsoft, aimed at enabling large-model training. Higher ZeRO stages (1→3) and enabling CPU offload reduce GPU VRAM usage, but might cause slower training.

To enable DeepSpeed, create a separate Deepspeed config in the **[storage shared across nodes](https://docs.anyscale.com/configuration/storage#shared)**. and reference it from your main training yaml config with:

```yaml
deepspeed: /mnt/cluster_storage/ds_z3_config.json
```

Below is a sample ZeRO-3 config:

`ds_z3_config.json`
```json
{
"train_batch_size": "auto",
"train_micro_batch_size_per_gpu": "auto",
"gradient_accumulation_steps": "auto",
"gradient_clipping": "auto",
"zero_allow_untested_optimizer": true,
"fp16": {
    "enabled": "auto",
    "loss_scale": 0,
    "loss_scale_window": 1000,
    "initial_scale_power": 16,
    "hysteresis": 2,
    "min_loss_scale": 1
},
"bf16": {
    "enabled": "auto"
},
"zero_optimization": {
    "stage": 3,
    "overlap_comm": false,
    "contiguous_gradients": true,
    "sub_group_size": 1e9,
    "reduce_bucket_size": "auto",
    "stage3_prefetch_bucket_size": "auto",
    "stage3_param_persistence_threshold": "auto",
    "stage3_max_live_parameters": 1e9,
    "stage3_max_reuse_distance": 1e9,
    "stage3_gather_16bit_weights_on_model_save": true
}
}
```

For a more detailed guide on acceleration and optimization methods including DeepSpeed on Ray, see [Speed and memory optimizations](https://docs.anyscale.com/llm/fine-tuning/speed-and-memory-optimizations).

In [3]:
%%bash
# Create a copy of the DeepSpeed configuration file in /mnt/cluster_storage
cp ../deepspeed-configs/ds_z3_config.json /mnt/cluster_storage/

## Step 4: Train and monitor

**Note**: If you installed Weights & Biases, set `WANDB_API_KEY` in the runtime environment. Otherwise, set `report_to: none` in `sft_lora_deepspeed.yaml` to avoid `api_token not set` errors.

With all configurations in place, you can launch fine-tuning or post-training in one of two ways:

### Option A: Run from a workspace (quick start)

The `USE_RAY=1` prefix tells LLaMA-Factory to run in distributed mode on the Ray cluster attached to your workspace.

In [5]:
%%bash
USE_RAY=1 llamafactory-cli train ../train-configs/sft_lora_deepspeed.yaml

[2026-02-08 08:51:32,783] [WARNING] [real_accelerator.py:209:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.
[2026-02-08 08:51:32,783] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cpu (auto detect)
INFO 02-08 08:51:34 [__init__.py:248] No platform detected, vLLM is running on UnspecifiedPlatform
WARNING 02-08 08:51:34 [_custom_ops.py:20] Failed to import from vllm._C with ImportError('libcuda.so.1: cannot open shared object file: No such file or directory')


2026-02-08 08:51:37,024	INFO worker.py:1747 -- Connecting to existing Ray cluster at address: 10.128.5.218:6379...
2026-02-08 08:51:37,035	INFO worker.py:1918 -- Connected to Ray cluster. View the dashboard at https://session-c1mvc6t862zj4fbguuknngnrgv.i.anyscaleuserdata.com 
2026-02-08 08:51:37,037	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_1598f5ff1c5690e55af864dcf6517685e149d19c.zip' (0.07MiB) to Ray cluster...
2026-02-08 08:51:37,037	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_1598f5ff1c5690e55af864dcf6517685e149d19c.zip'.



View detailed results here: /mnt/cluster_storage/qwen2.5_32b_lora_sft
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2026-02-08_07-42-51_475236_185/artifacts/2026-02-08_08-51-37/qwen2.5_32b_lora_sft/driver_artifacts`
(TrainTrainable pid=2869, ip=10.128.7.103) [2026-02-08 08:51:45,441] [WARNING] [real_accelerator.py:209:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.
(TrainTrainable pid=2869, ip=10.128.7.103) [2026-02-08 08:51:45,442] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cpu (auto detect)

Training started with configuration:
╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Training config                                                                                              │
├──────────────────────────────────────────────────────────────────────────────────────────────────

(RayTrainWorker pid=3001, ip=10.128.7.103) Setting up process group for: env:// [rank=0, world_size=4]
(TorchTrainer pid=2869, ip=10.128.7.103) Started distributed worker processes: 
(TorchTrainer pid=2869, ip=10.128.7.103) - (node_id=c0a063c3a2d8319e8a19333c768480af090e91c460068963ebf9fb27, ip=10.128.7.103, pid=3001) world_rank=0, local_rank=0, node_rank=0
(TorchTrainer pid=2869, ip=10.128.7.103) - (node_id=c0a063c3a2d8319e8a19333c768480af090e91c460068963ebf9fb27, ip=10.128.7.103, pid=3002) world_rank=1, local_rank=1, node_rank=0
(TorchTrainer pid=2869, ip=10.128.7.103) - (node_id=c0a063c3a2d8319e8a19333c768480af090e91c460068963ebf9fb27, ip=10.128.7.103, pid=3003) world_rank=2, local_rank=2, node_rank=0
(TorchTrainer pid=2869, ip=10.128.7.103) - (node_id=c0a063c3a2d8319e8a19333c768480af090e91c460068963ebf9fb27, ip=10.128.7.103, pid=3000) world_rank=3, local_rank=3, node_rank=0


(RayTrainWorker pid=3001, ip=10.128.7.103) [2026-02-08 08:51:56,572] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)
(RayTrainWorker pid=3001, ip=10.128.7.103) [2026-02-08 08:52:00,532] [INFO] [comm.py:669:init_distributed] cdb=None
(RayTrainWorker pid=3001, ip=10.128.7.103) [INFO|2026-02-08 08:52:00] llamafactory.hparams.parser:143 >> Set `ddp_find_unused_parameters` to False in DDP training since LoRA is enabled.
(RayTrainWorker pid=3001, ip=10.128.7.103) [INFO|2026-02-08 08:52:00] llamafactory.hparams.parser:406 >> Process rank: 0, world size: 4, device: cuda:0, distributed training: True, compute dtype: torch.bfloat16


...

(RayTrainWorker pid=3001, ip=10.128.7.103) [INFO|trainer.py:3993] 2026-02-08 09:02:55,205 >> Saving model checkpoint to qwen2.5_32b_lora_sft
(RayTrainWorker pid=3000, ip=10.128.7.103) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/mnt/cluster_storage/qwen2.5_32b_lora_sft/TorchTrainer_6ecdb_00000_0_2026-02-08_08-51-37/checkpoint_000001) [repeated 3x across cluster]
(RayTrainWorker pid=3001, ip=10.128.7.103) [INFO|configuration_utils.py:698] 2026-02-08 09:02:55,432 >> loading configuration file config.json from cache at /home/ray/.cache/huggingface/hub/models--Qwen--Qwen2.5-32B-Instruct/snapshots/5ede1c97bbab6ce5cda5812749b4c0bdf79b18dd/config.json
(RayTrainWorker pid=3001, ip=10.128.7.103) [INFO|configuration_utils.py:770] 2026-02-08 09:02:55,432 >> Model config Qwen2Config {
(RayTrainWorker pid=3001, ip=10.128.7.103)   "architectures": [
(RayTrainWorker pid=3001, ip=10.128.7.103)     "Qwen2ForCausalLM"
(RayTrainWorker pid=3001, ip=10.128.7.103)   ],
(RayTrainWor

(RayTrainWorker pid=3001, ip=10.128.7.103) ***** train metrics *****
(RayTrainWorker pid=3001, ip=10.128.7.103)   epoch                    =        3.0
(RayTrainWorker pid=3001, ip=10.128.7.103)   total_flos               =    68936GF
(RayTrainWorker pid=3001, ip=10.128.7.103)   train_loss               =     0.3662
(RayTrainWorker pid=3001, ip=10.128.7.103)   train_runtime            = 0:10:02.93
(RayTrainWorker pid=3001, ip=10.128.7.103)   train_samples_per_second =      1.493
(RayTrainWorker pid=3001, ip=10.128.7.103)   train_steps_per_second   =      0.095


(RayTrainWorker pid=3001, ip=10.128.7.103) [INFO|modelcard.py:450] 2026-02-08 09:02:55,941 >> Dropping the following result as it does not have all the necessary fields:
(RayTrainWorker pid=3001, ip=10.128.7.103) {'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}


(RayTrainWorker pid=3001, ip=10.128.7.103) Figure saved at: qwen2.5_32b_lora_sft/training_loss.png
(RayTrainWorker pid=3001, ip=10.128.7.103) [WARNING|2026-02-08 09:02:55] llamafactory.extras.ploting:148 >> No metric eval_loss to plot.
(RayTrainWorker pid=3001, ip=10.128.7.103) [WARNING|2026-02-08 09:02:55] llamafactory.extras.ploting:148 >> No metric eval_accuracy to plot.

Training completed after 2 iterations at 2026-02-08 09:02:58. Total running time: 11min 20s


2026-02-08 09:02:58,095	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/mnt/cluster_storage/qwen2.5_32b_lora_sft' in 0.0139s.


### Option B: Run as an Anyscale job (production)

For longer or production runs, submit the training as an **Anyscale job**. Jobs run outside your interactive session for better stability, retries, and durable logs. You package LLaMA-Factory and other libraries in a container image and launch with a short job config. See [Run LLaMA-Factory as an Anyscale job](https://docs.anyscale.com/llm/fine-tuning/llamafactory-jobs) for the step-by-step guide.

### Monitoring with Weights & Biases (WandB)
If you enabled Weights & Biases (with `report_to: wandb` in the training config YAML file), you can monitor your training job in real-time. Look for the training loss to decrease steadily, which indicates the model is learning.

**Weights & Biases example**

![WandB](https://anyscale-public-materials.s3.us-west-2.amazonaws.com/llm-finetuning/llama-factory/3.2.1/3.2.1-wandb.png)

For a more detailed guide on tracking experiments with other tools such as Weights & Biases or MLflow, see [Observability and tracking](https://docs.anyscale.com/llm/fine-tuning/observability-and-tracking).

## Step 5: Locate checkpoints

Ray Train writes checkpoints under `ray_storage_path/ray_run_name`. In this example run, the path is: `/mnt/cluster_storage/qwen2.5_32b_lora_sft`. 

Inside, you see a **trainer session** directory named like:
`TorchTrainer_8c6a5_00000_0_2025-09-09_09-53-45/`.

- Ray Train creates `TorchTrainer_*` **when the trainer starts**; the suffix encodes a short run ID and the **start timestamp**.
- Within that directory, Ray Train names checkpoints `checkpoint_000xxx/`, where the number is the saved ordered checkpoints.

Control the save cadence with `save_strategy` and `save_steps`. For instructions on how to resume interrupted training with `resume_from_checkpoint` and more, see [Understand the artifacts directory](https://docs.anyscale.com/llm/fine-tuning/checkpointing#artifacts-directory).

## Step 6: Export the model

If you use LoRA, you can keep the base model and adapters separate for [multi-LoRA deployment](https://docs.anyscale.com/llm/serving/multi-lora) or [merge the adapters](https://docs.anyscale.com/llm/fine-tuning/checkpointing#merge-lora) into the base model for low-latency inference. 

For full fine-tuning or freeze-tuning, export the fine-tuned model directly.

You may optionally apply [post-training quantization](https://docs.anyscale.com/llm/fine-tuning/checkpointing#ptq) on merged or full models before serving.